In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver 

In [2]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [3]:
llm = ChatOpenAI()


def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']

    # send to llm
    response = llm.invoke(messages)

    # response store state
    return {'messages': [response]}

In [7]:
checkpointer = MemorySaver()
graph = StateGraph(ChatState)

# add nodes
graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)

In [8]:
initial_state = {
    'messages': [HumanMessage(content='What is the capital of india')]
}

chatbot.invoke(initial_state)['messages'][-1].content

ValueError: Checkpointer requires one or more of the following 'configurable' keys: thread_id, checkpoint_ns, checkpoint_id

In [9]:
thread_id = '1'
while True:
    user_message = input("You: ")
    print("User:", user_message)

    if(user_message.strip().lower() in ['exit', 'quit','bye']):
        break

    config={'configurable': {'thread_id': thread_id}}
    state = {
        'messages': [HumanMessage(content=user_message)]
    }
    response = chatbot.invoke(state,config=config)
    print("AI:", response['messages'][-1].content)

User: hi my name is raj
AI: Hello Raj, nice to meet you! How can I assist you today?
User: what is my name
AI: Your name is Raj. Is there anything else I can help you with?
User: multiply 2 by 48
AI: 2 multiplied by 48 is equal to 96.
User: now subtract 27 from it
AI: If you subtract 27 from 96, you get 69.
User: exit


In [11]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='hi my name is raj', additional_kwargs={}, response_metadata={}, id='ec629d53-bdf4-4d77-a023-eecc2844842b'), AIMessage(content='Hello Raj, nice to meet you! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 12, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CC0rnlWZGdZHVK9RbydK1kCbOhoEg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--28bea7f8-0212-402a-83c6-8b5ec87a8edf-0', usage_metadata={'input_tokens': 12, 'output_tokens': 15, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning':